In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
from huggingface_hub import login

login()

In [2]:
!pip install langchain langchain-community langchain-core transformers pypdf sentence-transformers transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 23.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.5 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 58.1 MB/s eta 0:00:00:00:0100:01


In [4]:
!pip -q install --upgrade pip
!pip -q install langchain langchain-community sentence-transformers faiss-cpu \
                transformers accelerate bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 22.3 MB/s eta 0:00:0000:0100:01


In [5]:
pip install fastapi uvicorn pyngrok transformers accelerate -q

Note: you may need to restart the kernel to use updated packages.


In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch, os

model_name = "mistralai/Mistral-7B-Instruct-v0.2"  
bnb = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16
)         

token = os.environ.get("HF_TOKEN", None)

tok = AutoTokenizer.from_pretrained(model_name, token=token)
tok.pad_token = tok.eos_token                        

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb,                        
    device_map="auto",
    token=token
).eval()


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [7]:
import json, re
import torch

@torch.inference_mode()
def chat_json_only(system, user, max_new_tokens=256):
    # deterministic generation for structure
    messages = [
        {"role": "system", "content":
         "Return ONLY a single valid JSON object. No prose, no markdown, no code fences."},
        {"role": "user", "content": user}
    ]
    # allow caller to override/add to system
    if system:
        messages[0]["content"] = system

    prompt = tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tok([prompt], return_tensors="pt").to(model.device)

    out = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,              
        temperature=0.0,               
        pad_token_id=tok.eos_token_id,
        eos_token_id=tok.eos_token_id
    )
    text = tok.decode(out[0], skip_special_tokens=True)
    return text

def extract_json(s: str):
    """grab the first top-level JSON object from text"""
    m = re.search(r'\{.*\}', s, flags=re.S)
    if not m:
        raise ValueError("No JSON object found")
    return json.loads(m.group(0))


In [8]:
system = (
  "Return ONLY a single valid JSON object. "
  "Keys: answer (string), tokens (integer). "
  "No prose, no markdown, no code fences."
)
user = "What is LangChain?"
raw = chat_json_only(system, user, max_new_tokens=200)
print("RAW:", raw)


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


RAW: [INST] Return ONLY a single valid JSON object. Keys: answer (string), tokens (integer). No prose, no markdown, no code fences.

What is LangChain? [/INST] {
  "answer": "LangChain is a hypothetical concept and I couldn't find specific information about it in my current knowledge base. It might be a potential combination of 'Language' and 'Blockchain' technologies, but without more context, it's impossible to provide an accurate definition.",
  "tokens": 12
}


In [9]:
import json, os
from pathlib import Path

DATA_DIR = "/kaggle/input/destinations"
Path(DATA_DIR).mkdir(parents=True, exist_ok=True)

In [10]:
import glob, json
from pathlib import Path
from langchain.docstore.document import Document
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

EMBED = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

def load_docs(data_dir: str):
    docs = []
    for fp in glob.glob(f"{data_dir}/*.json"):
        items = json.load(open(fp, "r", encoding="utf-8"))
        for it in items:
            content = (
                f"City: {it['city']}\n"
                f"Name: {it['name']}\n"
                f"Tags: {', '.join(it.get('tags', []))}\n"
                f"Best time: {it.get('best_time','')}\n"
                f"Area: {it.get('area','')}\n"
                f"Avg price (USD): {it.get('avg_price_usd',0)}\n"
                f"Duration (h): {it.get('duration_hours',2)}\n"
                f"Notes: {it.get('notes','')}\n"
            )
            docs.append(Document(page_content=content, metadata=it))
    return docs

INDEX_DIR = "/kaggle/working/index/faiss"
Path(INDEX_DIR).parent.mkdir(parents=True, exist_ok=True)

docs = load_docs(DATA_DIR)
vs = FAISS.from_documents(docs, EMBED)
vs.save_local(INDEX_DIR)

len(docs), "index saved to", INDEX_DIR


/tmp/ipykernel_36/3049898941.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  EMBED = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

(150, 'index saved to', '/kaggle/working/index/faiss')

In [11]:
from langchain_community.vectorstores import FAISS

# reload index (simulates fresh start)
vs = FAISS.load_local(INDEX_DIR, EMBED, allow_dangerous_deserialization=True)
# when you build the retriever
retriever = vs.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 15,         
        "fetch_k": 60,    
        "lambda_mult": 0.8
    }
)


query = "Dubai attractions to visit"
hits = retriever.get_relevant_documents(query)

print("Top results:", len(hits))
for i, d in enumerate(hits, 1):
    print(f"\n[{i}] ---")
    print(d.page_content.split("\n", 3)[0:3])  # first few lines


Top results: 15

[1] ---
['City: Dubai', 'Name: Dubai Frame', 'Tags: view, architecture']

[2] ---
['City: Dubai', 'Name: Etihad Museum', 'Tags: history, museum']

[3] ---
['City: Dubai', 'Name: Desert Safari (with dinner)', 'Tags: adventure, culture, food']

[4] ---
['City: Dubai', 'Name: Dubai Mall & Fountain Show', 'Tags: mall, show, family']

[5] ---
['City: Dubai', 'Name: Abra Ride across Dubai Creek', 'Tags: boat, view, heritage']

[6] ---
['City: Dubai', 'Name: Al Seef Waterfront', 'Tags: walk, heritage, food']

[7] ---
['City: Dubai', 'Name: Al Fahidi Historical Neighbourhood', 'Tags: heritage, walk, market']

[8] ---
['City: Dubai', 'Name: Museum of the Future (outside/inside)', 'Tags: museum, tech']

[9] ---
['City: Dubai', 'Name: Gold & Spice Souks', 'Tags: market, walk']

[10] ---
['City: Dubai', 'Name: Jumeirah Mosque (visit)', 'Tags: religious, culture']

[11] ---
['City: Dubai', 'Name: Madinat Jumeirah Canals Walk', 'Tags: walk, view, food']

[12] ---
['City: Dubai', 'Na

/tmp/ipykernel_36/1757137474.py:17: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  hits = retriever.get_relevant_documents(query)


In [12]:
from pydantic import BaseModel, Field, ValidationError
from typing import List

class Slot(BaseModel):
    place: str
    reason: str
    est_cost_usd: float = Field(ge=0)

class DayPlan(BaseModel):
    city: str
    day_number: int
    morning: Slot
    afternoon: Slot
    evening: Slot
    daily_total_usd: float = Field(ge=0)

class Itinerary(BaseModel):
    days: List[DayPlan]


In [13]:

PLANNER_SYSTEM = (
    "You are a helpful travel planner. Use ONLY the retrieved attractions; do not invent new places. "
    "Return ONLY one valid JSON object. Keys: days (list). Each day has: city (string), day_number (int), "
    "morning/afternoon/evening (each with: place, reason, est_cost_usd (float)), daily_total_usd (float). "
    "Ensure daily_total_usd ≈ sum of the three slots. Keep the total within the user's budget."
    "Across the ENTIRE trip, DO NOT REPEAT any place name. If a slot would repeat, pick a different place or use a free explore suggestion."
    
)

PLANNER_USER_TMPL = (
    "Destination: {destination}\n"
    "Days: {days}\n"
    "Total budget (USD): {budget}\n"
    "Preferences: {preferences}\n\n"
    "Retrieved facts:\n{context}\n"
)


In [14]:
def build_context(destination: str, k: int = 12) -> str:
    # Prefer retriever.invoke if you upgraded; your current .get_relevant_documents is fine:
    hits = retriever.get_relevant_documents(f"{destination} attractions")
    blocks = []
    for d in hits[:k]:
        md = d.metadata
        blocks.append(
            f"{md.get('city')} : {md.get('name')} | "
            f"tags={','.join(md.get('tags', []))} | "
            f"best_time={md.get('best_time','')} | area={md.get('area','')} | "
            f"avg_price_usd={md.get('avg_price_usd',0)} | duration_h={md.get('duration_hours',2)}"
        )
    return "\n".join(blocks) if blocks else "No retrieved items."

@torch.inference_mode()
def plan_itinerary(destination: str, days: int, budget: float, preferences: str):
    context = build_context(destination, k=12)
    user_msg = PLANNER_USER_TMPL.format(
        destination=destination, days=days, budget=budget,
        preferences=preferences, context=context
    )
    raw = chat_json_only(PLANNER_SYSTEM, user_msg, max_new_tokens=600)
    data = extract_json(raw)            
    iti  = Itinerary(**data)            
    return iti                          


In [15]:
import pandas as pd

destination = "dubai"
days = 2
budget = 400
prefs = "history, food, markets"

iti = plan_itinerary(destination, days, budget, prefs)

rows = []
for d in iti.days:
    rows.append({
        "Day": d.day_number,
        "Morning":   f"{d.morning.place} — {d.morning.reason} (${d.morning.est_cost_usd:.2f})",
        "Afternoon": f"{d.afternoon.place} — {d.afternoon.reason} (${d.afternoon.est_cost_usd:.2f})",
        "Evening":   f"{d.evening.place} — {d.evening.reason} (${d.evening.est_cost_usd:.2f})",
        "Daily Total (USD)": round(d.daily_total_usd, 2),
    })
df = pd.DataFrame(rows).sort_values("Day")

from IPython.display import display
display(df.style.set_properties(**{"white-space": "pre-wrap"}))


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


,Day,Morning,Afternoon,Evening,Daily Total (USD)
0,1,"Al Fahidi Historical Neighbourhood — heritage, market ($0.00)","Etihad Museum — history, museum ($8.00)","Al Seef Waterfront — walk, heritage, food ($0.00)",8.000000
1,2,"Dubai Frame — view, architecture ($14.00)","Desert Safari (with dinner) — adventure, culture, food ($60.00)","Burj Khalifa At the Top — view, landmark ($45.00)",127.000000


In [16]:
import os, socket, threading, time
from fastapi import FastAPI, Request, HTTPException
from pydantic import BaseModel, Field
import uvicorn
from pyngrok import ngrok, conf
import json

# === CONFIG ===
API_KEY = "secret123"                     
NGROK_TOKEN = os.environ.get("NGROK_TOKEN","32MzUMaST5RzRWwaNAeyQrQMgtM_463mXFbwi7NTNrgRvdGVc")  

try:
    app
except NameError:
    app = FastAPI()


    

In [18]:
from typing import Optional, Dict
from pydantic import BaseModel, Field
from fastapi import FastAPI, Request, HTTPException


class PlanRequest(BaseModel):
    city: str = Field(..., example="Cairo")
    days: int = Field(..., ge=1, le=14, example=3)
    budget: float = Field(..., ge=0, example=400)
    preferences: str = Field("", example="history, food, markets")
    filters: Optional[Dict] = None
    constrain_to_budget: bool = False  # keep if you use the budget nudge

def _to_dict(x):
    """Return a plain dict whether x is already a dict or a Pydantic model."""
    if isinstance(x, dict):
        return x
    if hasattr(x, "model_dump"):   # pydantic v2
        return x.model_dump()
    if hasattr(x, "dict"):         # pydantic v1
        return x.dict()
    return x

@app.post("/plan")
def plan(req: PlanRequest, request: Request):
    # 1) Auth
    if request.headers.get("authorization") != f"Bearer {API_KEY}":
        raise HTTPException(status_code=401, detail="Unauthorized")

    # 2) Optional budget nudge into preferences (light hint for the planner)
    prefs = req.preferences
    if req.constrain_to_budget:
        prefs += ", IMPORTANT: keep daily total under budget; prefer free/cheap options when needed"

    # 3) Call planner and normalize to dict
    out = plan_itinerary(req.city, req.days, req.budget, prefs)
    data = _to_dict(out)
    if not isinstance(data, dict) or "days" not in data:
        raise HTTPException(status_code=500, detail="Planner returned invalid data")

    # 4) Optional toy filters (safe/defensive)
    flt = req.filters or {}
    if flt.get("vegetarian"):
        for d in data["days"]:
            try:
                if "restaurant" in d["evening"]["reason"].lower():
                    d["evening"]["reason"] += " (vegetarian option)"
            except KeyError:
                pass

    if flt.get("family_friendly"):
        for d in data["days"]:
            try:
                if "walk" in d["evening"]["reason"].lower():
                    d["evening"]["reason"] += " (family-friendly)"
            except KeyError:
                pass

    if flt.get("avoid_long_walks"):
        for d in data["days"]:
            try:
                if "walk" in d["afternoon"]["reason"].lower():
                    d["afternoon"]["reason"] = d["afternoon"]["reason"].replace("walk", "short visit")
            except KeyError:
                pass

    # 5) Return final JSON
    return data


In [21]:
def _free_port():
    s = socket.socket()
    s.bind(('', 0))
    port = s.getsockname()[1]
    s.close()
    return port

port = _free_port()

ngrok.kill()

conf.get_default().auth_token = NGROK_TOKEN
public_url = ngrok.connect(port, "http").public_url

print("PUBLIC URL:", public_url)
print("AUTH HEADER: Authorization: Bearer", API_KEY)
print("Endpoint:", f"{public_url}/plan")

def _run():
    uvicorn.run(app, host="0.0.0.0", port=port, log_level="info")

thread = threading.Thread(target=_run, daemon=True)
thread.start()
time.sleep(1)

PUBLIC URL: https://8eeb555ef2c9.ngrok-free.app
AUTH HEADER: Authorization: Bearer secret123
Endpoint: https://8eeb555ef2c9.ngrok-free.app/plan


INFO:     Started server process [36]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:39965 (Press CTRL+C to quit)
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


INFO:     154.180.31.203:0 - "POST /plan HTTP/1.1" 200 OK


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


INFO:     154.180.31.203:0 - "POST /plan HTTP/1.1" 200 OK


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


INFO:     154.180.31.203:0 - "POST /plan HTTP/1.1" 200 OK


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


INFO:     154.180.31.203:0 - "POST /plan HTTP/1.1" 200 OK
